# 02_ocetrac.ipynb
### Use Ocetrac to identify and track MCS objects 

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from ocetrac import model as oce

#### Import data files

In [ ]:
file_path_list = ('/burg/abernathey/users/hillary/oisst_v21/negative_extremes_OISSTv21.nc',
                  '/burg/abernathey/users/hillary/oisst_v21/preprocessed_OISSTv21.nc')

ds = xr.open_mfdataset(file_path_list)
ds

#### [Optional] Adjust land mask to ignore points poleward of 65ºN and 70ºS

In [ ]:
mask = ds.lsmask.where((ds.lat<65) & (ds.lat>-70), drop=False, other=0) 
mask.plot()

### Run Ocetrac

In [ ]:
# Set model parameters
da = ds.mcs_ssta_notrend.load() # load the DataArray into memory
radius = 8 # radius for structuring element
min_size_quartile = 0.75 # threshold for object areas
timedim = 'time'
xdim = 'lon'
ydim = 'lat'

In [ ]:
Tracker =  oce.Tracker(da, mask, radius, min_size_quartile, timedim, xdim, ydim)

In [ ]:
%%time 
blobs = Tracker.track()

**The basic output of `Tracker.track` provides:**
- the minimum object area used to filter out MCS that are smaller than this threshold. Area is computed for all objects at each time step. `Ocetrac` looks at the global distribution of object area and finds the size that corresponds to a certain percentile defined by `area_quantile`, which can range from 0.0 to 1.0. Larger values for `area_quantile` exclude more MCS objects and will increase the minimum object area threshold. 
- inital features identified have undergone size thresholding, however are not connected in time. 
- final features tracked are the number of unique MCS objects tracked in space and time. 

#### Let's make a quick plot

In [ ]:
from matplotlib.colors import ListedColormap
maxl = int(np.nanmax(blobs.values))
cm = ListedColormap(np.random.random(size=(maxl, 3)).tolist())

# MCS identified with Ocetrac
blobs.sel(time=('2012-08-01')).plot(cmap= cm)

### Save tracked labels

In [ ]:
# Specify path to save output 
path = '/burg/abernathey/users/hillary/oisst_v21/'

ds_out = blobs.to_dataset(name='ocetrac_labels')
ds_out.attrs['data'] = 'OISST v2.1 no trend'

# Save Dataset to netCDF
ds_out.to_netcdf(path+'ocetrac_MCS_notrend.nc', mode='w')